In [24]:
vets = pd.read_csv('../data/vets.csv', encoding = "ISO-8859-1")

In [25]:
vets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885 entries, 0 to 884
Data columns (total 45 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Id                                       885 non-null    object 
 1   AccountId                                885 non-null    object 
 2   Title                                    5 non-null      object 
 3   Birthdate                                79 non-null     object 
 4   OwnerId                                  885 non-null    object 
 5   CreatedDate                              885 non-null    object 
 6   CreatedById                              885 non-null    object 
 7   LastModifiedDate                         885 non-null    object 
 8   LastModifiedById                         885 non-null    object 
 9   SystemModstamp                           885 non-null    object 
 10  LastActivityDate                         500 non-n

##### Had to do some additional wrangling in Excel to get all the values to 0 for the columns where professionals were supposed to enter data, but not veterans

In [31]:
cols = vets.columns[37:44]
vets.drop(cols, inplace=True, axis=1)

In [32]:
vets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885 entries, 0 to 884
Data columns (total 38 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Id                                       885 non-null    object
 1   AccountId                                885 non-null    object
 2   Title                                    5 non-null      object
 3   Birthdate                                79 non-null     object
 4   OwnerId                                  885 non-null    object
 5   CreatedDate                              885 non-null    object
 6   CreatedById                              885 non-null    object
 7   LastModifiedDate                         885 non-null    object
 8   LastModifiedById                         885 non-null    object
 9   SystemModstamp                           885 non-null    object
 10  LastActivityDate                         500 non-null    objec

In [35]:
#drop non-essential "Title" column
vets.drop('Title', axis=1, inplace=True)

#### Since I am only focusing on the veteran attributes themselves, I'm dropping the admin data columns.  I have a previous version of this data set saved, so if necessary I can always go back for further analysis

In [42]:
vets.drop('AccountId', axis=1, inplace=True)

In [48]:
admin_cols = vets.columns[4:9]
vets.drop(admin_cols, axis=1, inplace=True)

In [52]:
vets.drop('OwnerId', axis=1, inplace=True)
vets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885 entries, 0 to 884
Data columns (total 30 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Id                                       885 non-null    object
 1   Birthdate                                79 non-null     object
 2   CreatedDate                              885 non-null    object
 3   Location                                 816 non-null    object
 4   City                                     798 non-null    object
 5   State                                    800 non-null    object
 6   How_did_you_hear_about_Elite_Meet        662 non-null    object
 7   Are_you_currently_seeking_to_join_Elite  652 non-null    object
 8   Your_transition_to_civilian_life_begin   553 non-null    object
 9   Undergrad                                747 non-null    object
 10  Undergraduate_School                     796 non-null    objec

#### Ok, I've got all the columns that I want to keep, now it's .fillna time!  For now it looks like I am working with 885 indiviudals with roughly 29 features.  This is approximately equal to the Elite Meet membership, which currently totals 1,040 members

In [56]:
#review what the birthday column looks like
vets['Birthdate'][vets['Birthdate'].notnull()].head(10)

5      3/14/89 0:00
6      9/29/83 0:00
7       7/4/92 0:00
8      2/26/77 0:00
9      4/12/75 0:00
10     12/8/88 0:00
11     1/11/82 0:00
12    10/22/88 0:00
13    12/25/75 0:00
14     9/29/91 0:00
Name: Birthdate, dtype: object

#### Most veteran applicatns did not list a birthdate, so I'll need to approximate most ages by using the "Years of Service" entry as a starting point

In [59]:
vets[(vets['Birthdate'].isnull()) & (vets['Years_of_Service'].isnull())].head()

,Id,Birthdate,CreatedDate,Location,City,State,How_did_you_hear_about_Elite_Meet,Are_you_currently_seeking_to_join_Elite,Your_transition_to_civilian_life_begin,Undergrad,Undergraduate_School,Undergrad_major,Grad_Degree,Graduate_School,Grad_Degree_Type,Years_of_Service,Military_specialty_in_real_words,Whichcourse,When_did_you_graduate,What_was_your_first_unit_after_grad,What_locations_do_you_WANT_to_live_in,Where_do_you_NOT_WANT_to_live,What_would_be_your_ideal_next_job,Target_Industries_Pls_pick_up_to_Three,of_informational_interviews_completed,of_job_formal_interviews_completed,of_counting_getting_a_job_through_EM,What_s_more_important_to_you,How_mobile_are_you_and_your_family,Lead_RecordType
1,003f400000cZo1RAAS,NaN,10/16/18 15:40,TX,NaN,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Crewman Qualification Training (SWCC),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Veteran
3,003f400000cZo3EAAS,NaN,10/16/18 15:40,"Chattanooga, TN",Chattanooga,TN,NaN,NaN,NaN,NaN,NaN,NaN,Complete,NaN,MBA- Vandy,NaN,NaN,Q Course (SF),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Veteran
4,003f400000cZo3LAAS,NaN,10/16/18 15:40,"San Francisco,CA",San Francisco,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winged as a fighter pilot (Flight Officer),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Veteran
84,003f400000fo4ZTAAY,NaN,10/31/18 14:55,"Venice Beach,CA",Venice Beach,CA,Linkedin - SEE SHERI S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Veteran
116,003f400000oOLf8AAG,NaN,1/24/19 13:40,"Honolulu,HI",Honolulu,HI,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,Special Forces Medical Sergeant?,Q Course (SF),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Veteran


#### I will need to create a new column titled "Age" as this is an important metric that I want to analyze.  835 out of 885 entries have a "Years of Service" entry, so that only leaves 50 entries that will require some additional sleuth work.  As shown above, the real challenge will be approximating an age for those who did not enter a Birthdate and did not give Years of Service.  Fortunately some listed the year that they graduated from their service school, which can be used as another age approximator.  

In [67]:
vets['Years_of_Service'][vets['Years_of_Service'].apply(lambda x: len(str(x)) > 2)].head(25)

1                                       NaN
3                                       NaN
4                                       NaN
7                                      Four
30                               Twenty Two
43                                      9.5
44                                      NaN
45                                      23+
55                                      28+
57                                      8.5
60                7 (at date of transition)
61     11 (will be at 12 when I transition)
62       16+ (12+ active,currently reserve)
69                       12 years,10 months
77                  19 (20 on 01 June 2019)
79                                      18+
81                                      9.5
83                                      9.5
84                                      NaN
90                                      3.5
94                                      7.5
103                                     6.5
116                             

#### Here again we can see the cons of allowing free form data entry on something as simple as Years of Service.  Given the mess, it's back to Excel for additional cleaning to remove all non-integer/float values.

In [68]:
vets.to_csv('../data/vets2.csv', index=False, header=True)